In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from time import time_ns
from PIL import Image
from functools import partial
from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

from yoeo.main import get_dv2_model, get_upsampler_and_expr, get_hr_feats
from yoeo.utils import load_image, convert_image, to_numpy, closest_crop, Experiment, do_2D_pca, add_flash_attention

from yoeo.comparisons.lift import ViTLiFTExtractor
from yoeo.comparisons.strided import StridedDv2

from typing import Callable

SEED = 10672
np.random.seed(SEED)
torch.manual_seed(SEED)

In [2]:
torch.cuda.empty_cache()

DEVICE = "cuda:0"

In [3]:
def measure_mem_time(
    device: torch.device | str,
    featurise_fn: Callable
) -> tuple[float, float]:
    torch.cuda.reset_peak_memory_stats(device)  # s.t memory is accurate
    torch.cuda.synchronize(device)  # s.t time is accurate

    def _to_MB(x: int) -> float:
        return x / (1024**2)

    def _to_s(t: int) -> float:
        return t / 1e9

    start_m = torch.cuda.max_memory_allocated(device)
    start_t = time_ns()

    featurise_fn()

    end_m = torch.cuda.max_memory_allocated(device)
    torch.cuda.synchronize(device)
    end_t = time_ns()

    return _to_MB(end_m - start_m), _to_s(end_t - start_t)

def rescale(arr: np.ndarray, swap_channels: bool=True) -> np.ndarray:
    if swap_channels:
        arr = np.transpose(arr, (1, 2, 0))
    h, w, c = arr.shape
    flat = arr.reshape((h * w, c))
    rescaled_flat = MinMaxScaler(clip=True).fit_transform(flat)
    return rescaled_flat.reshape((h, w, c))

In [4]:
normal_dv2 = get_dv2_model(fit_3d=False, device=DEVICE)
dv2 = get_dv2_model(True, device=DEVICE)

model_path = "../trained_models/e5000_full_fit_reg.pth"
cfg_path = "../yoeo/models/configs/combined_no_shift.json"

upsampler, expr = get_upsampler_and_expr(model_path, cfg_path, device=DEVICE)
# upsampler = upsampler.half()

Using cache found in /home/ronan/.cache/torch/hub/facebookresearch_dinov2_main
Using cache found in /home/ronan/.cache/torch/hub/ywyue_FiT3D_main


[128, 128, 128, 128]


In [5]:
featup_jbu = torch.hub.load("mhamilton723/FeatUp", "dinov2", use_norm=True).to(DEVICE).eval()

Using cache found in /home/ronan/.cache/torch/hub/mhamilton723_FeatUp_main
Using cache found in /home/ronan/.cache/torch/hub/facebookresearch_dinov2_main


In [6]:
strided = StridedDv2('dinov2', 'vits14_reg', 1).to(DEVICE).eval()
strided.model = add_flash_attention(strided.model)

Using cache found in /home/ronan/.cache/torch/hub/facebookresearch_dinov2_main


In [7]:
lift_path = "../trained_models/lift/lift_dino_vits8.pth"
lift = ViTLiFTExtractor('dino_vits8', lift_path=lift_path, channel=384, facet='key')
# lift.extractor.model = add_flash_attention(lift.extractor.model)

Using cache found in /home/ronan/.cache/torch/hub/facebookresearch_dino_main


Loaded Backbone: dino_vits8
Loaded LiFT module from: ../trained_models/lift/lift_dino_vits8.pth


In [8]:
torch_hub_name = "loftup_dinov2s_reg"
lu_upsampler = torch.hub.load("andrehuang/loftup", torch_hub_name, pretrained=True)
lu_upsampler = lu_upsampler.to(DEVICE)

Using cache found in /home/ronan/.cache/torch/hub/andrehuang_loftup_main


In [9]:
PATH = "fig_data/perf_landscape/church_compare.png"
img = Image.open(PATH).convert("RGB")

In [27]:
@torch.no_grad()
def _our_featurise(img: Image.Image, dv2: torch.nn.Module, upsampler: torch.nn.Module, expr: Experiment) -> np.ndarray:
    hr_feats = get_hr_feats(img, dv2, upsampler, DEVICE, n_ch_in=expr.n_ch_in)
    hr_feats_np = to_numpy(hr_feats)
    reduced_hr = hr_feats_np[:3]
    return reduced_hr

@torch.no_grad()
def _original_featurise(img: Image.Image, dv2: torch.nn.Module) -> np.ndarray:
    # _img = Image.open(path).convert("RGB")
    _h, _w = img.height, img.width
    tr = closest_crop(_h, _w)

    tensor = convert_image(img, tr)
    _, _, h, w = tensor.shape
    with torch.autocast("cuda", torch.float16):
        dino_feats = dv2.forward_features(tensor)['x_norm_patchtokens']
    n_patch_w, n_patch_h = w // 14, h // 14
    dino_feats = dino_feats.permute((0, 2, 1))
    dino_feats = dino_feats.reshape((1, -1, n_patch_h, n_patch_w,))
    dino_feats_np = to_numpy(dino_feats)
    return dino_feats_np

@torch.no_grad()
def _jbu_featurise(img: Image.Image, jbu: torch.nn.Module) -> np.ndarray:
    # _img = Image.open(path).convert("RGB")
    _h, _w = img.height, img.width
    tr = closest_crop(_h, _w)

    tensor = convert_image(img, tr)
    with torch.autocast("cuda", torch.float16):
        jbu_feats = jbu(tensor.to(torch.float32))
    jbu_feats = F.interpolate(jbu_feats, (_h, _w))
    jbu_feats_np = to_numpy(jbu_feats)
    return jbu_feats_np[:3]

@torch.no_grad()
def _lift_featurise(path: str, img: Image.Image, lift: ViTLiFTExtractor, n: int=3, patch_size: int=8) -> np.ndarray:
    # _img = Image.open(path).convert("RGB")
    _h, _w = img.height, img.width

    image_batch, _ = lift.preprocess(path, (_h, _w))
    image_batch = image_batch.to(DEVICE)

    with torch.autocast("cuda", torch.float16):
        lift_feats = lift.extract_descriptors_iterative_lift(image_batch, lift_iter=n)
    _, _, c = lift_feats.shape
    sf = int(patch_size / (2**n))
    reshaped = lift_feats.squeeze(0).T.reshape((c, _h // sf, _w //  sf))
    return to_numpy(reshaped)

@torch.no_grad()
def _strided_featurise(img: Image.Image, strided: StridedDv2) -> np.ndarray:
    # _img = Image.open(path).convert("RGB")
    _h, _w = img.height, img.width
    tr = closest_crop(_h, _w)

    tensor = convert_image(img, tr)
    with torch.autocast("cuda", torch.float16):
        strided_feats = strided(tensor)
    strided_feats = F.interpolate(strided_feats, (_h, _w))
    strided_feats_feats_np = to_numpy(strided_feats)
    return strided_feats_feats_np

@torch.no_grad()
def _loftup_featurise(img: Image.Image, dv2: torch.nn.Module, upsampler: torch.nn.Module) -> np.ndarray:
    _h, _w = img.height, img.width
    tr = closest_crop(_h, _w)
    tensor = convert_image(img, tr)
    _, _, h, w = tensor.shape
    lr_feats: torch.Tensor = dv2.forward_features(tensor)['x_norm_patchtokens']
    n_patch_w, n_patch_h = w // 14, h // 14
    lr_feats = lr_feats.permute((0, 2, 1))
    lr_feats = lr_feats.reshape((1, -1, n_patch_h, n_patch_w,))
    with torch.autocast("cuda", torch.float16):
        hr_feats = lu_upsampler(lr_feats, tensor) 
    
    feats = F.interpolate(hr_feats, (_h, _w))

    feats = to_numpy(feats)
    return feats

In [28]:
our_featurise = partial(_our_featurise, img=img, dv2=dv2, upsampler=upsampler, expr=expr)
original_featurise = partial(_original_featurise, img=img, dv2=normal_dv2)
jbu_featurise = partial(_jbu_featurise, img=img, jbu=featup_jbu)
lift_featurise = partial(_lift_featurise, path=PATH, img=img, lift=lift)
strided_featurise = partial(_strided_featurise, img=img, strided=strided)
loftup_featurise = partial(_loftup_featurise, img=img, dv2=normal_dv2, upsampler=upsampler)

fns = (our_featurise, jbu_featurise, lift_featurise, strided_featurise, loftup_featurise)
featuriser_names = ("Ours", "FeatUp (JBU)", "LiFT", "Strided", 'LoftUp')

In [29]:
# dry run
for i, fn in enumerate(fns):
    print(featuriser_names[i])
    mem, time = measure_mem_time(DEVICE, fn)

Ours
FeatUp (JBU)
LiFT
Strided
LoftUp


In [30]:
mem_time_results: dict[str, dict] = {}
N_REPEATS =5
for name, fn in zip(featuriser_names, fns):
    mems, times = [], []
    for i in range(N_REPEATS):
        mem, time = measure_mem_time(DEVICE, fn)
        # if name in ('Strided', 'LiFT'):
        #     # LiFT & strided upsample all 384 DINO features, JBU & Ours upsample to 128
        #     # for fair comparison we scale memory of those approaches with that ratio
        #     mem *= 128 / 384
        mems.append(mem)
        times.append(time)
    mem_time_results[name] = {'time': np.mean(times), 'mem': np.mean(mems)}
print(mem_time_results)

{'Ours': {'time': 0.10608810800000001, 'mem': 293.193359375}, 'FeatUp (JBU)': {'time': 0.14406022200000002, 'mem': 1039.6162109375}, 'LiFT': {'time': 0.0716691202, 'mem': 346.13818359375}, 'Strided': {'time': 2.2429712376, 'mem': 1113.39892578125}, 'LoftUp': {'time': 0.15695184199999998, 'mem': 2107.9921875}}


In [102]:
# lengths = list(range(224, 1344 - 64, 64))
lengths = list(range(224, 800, 112))

our_featurise_l = partial(_our_featurise, dv2=dv2, upsampler=upsampler, expr=expr)
jbu_featurise_l = partial(_jbu_featurise, jbu=featup_jbu)
lift_featurise_l = partial(_lift_featurise, path=PATH, lift=lift)
strided_featurise_l = partial(_strided_featurise, strided=strided)
loftup_featurise_l = partial(_loftup_featurise, dv2=normal_dv2, upsampler=upsampler)

fns_vs_l = (our_featurise_l, jbu_featurise_l, lift_featurise_l, strided_featurise_l, loftup_featurise_l)

mem_vs_l: dict[str, list[float]] = {}
time_vs_l: dict[str, list[float]] = {}

for name, fn in zip(featuriser_names, fns_vs_l):
    mems, times = [], []
    for l in lengths:
        torch.cuda.empty_cache()
        print(f"{name}: {l}")
        resized = img.resize((l, l))
        if name  == 'LoftUp' and l > 670:
            continue
        if name in ('FeatUp (JBU)', 'Strided') and l > 800:
            continue
        bound = partial(fn, img=resized)
        mem, time = measure_mem_time(DEVICE, bound)
        
        # if name in ('Strided', 'LiFT'):

        #     mem *= 128 / 384
        mems.append(mem)
        times.append(time)
    mem_vs_l[name] = mems
    time_vs_l[name] = times

print(mem_vs_l)
print(time_vs_l)

Ours: 224
Ours: 336
Ours: 448
Ours: 560
Ours: 672
Ours: 784
FeatUp (JBU): 224
FeatUp (JBU): 336
FeatUp (JBU): 448
FeatUp (JBU): 560
FeatUp (JBU): 672
FeatUp (JBU): 784
LiFT: 224
LiFT: 336
LiFT: 448
LiFT: 560
LiFT: 672
LiFT: 784
Strided: 224
Strided: 336
Strided: 448
Strided: 560
Strided: 672
Strided: 784
LoftUp: 224
LoftUp: 336
LoftUp: 448
LoftUp: 560
LoftUp: 672
LoftUp: 784
{'Ours': [130.71044921875, 290.13134765625, 513.94873046875, 802.20849609375, 1154.8046875, 1569.126953125], 'FeatUp (JBU)': [483.87939453125, 1036.48681640625, 1805.619140625, 2793.9111328125, 4002.5068359375, 5429.61474609375], 'LiFT': [176.0341796875, 346.08349609375, 577.62109375, 891.05322265625, 1787.07373046875, 3256.2421875], 'Strided': [499.53759765625, 1113.162109375, 1984.15673828125, 3113.04736328125, 4499.18798828125, 6141.45361328125], 'LoftUp': [572.8203125, 2108.99560546875, 5803.9755859375, 13198.03515625]}
{'Ours': [0.086826356, 0.106560184, 0.180681802, 0.270776957, 0.41630782, 0.576471766], 'Fea

In [103]:
original_pcaed = do_2D_pca(original_featurise(), 3)
im0 = rescale(original_pcaed, swap_channels=False)
print(im0.shape)

(24, 24, 3)


In [104]:
im1 = rescale(our_featurise())
im2 = rescale(jbu_featurise())
lift_feats = lift_featurise()
pcaed = do_2D_pca(lift_feats, 3)
im3 = rescale(pcaed, swap_channels=False).astype(np.float32)
stride_feats = strided_featurise()
strided_pcaed = do_2D_pca(stride_feats, 3)
im4 = rescale(strided_pcaed, swap_channels=False)
loft_feats = loftup_featurise()
loft_pca = do_2D_pca(loft_feats, 3)
im5 = rescale(loft_pca, swap_channels=False )

In [105]:
%%capture
imgs = (im1, im2, im3, im4, im5)
fig, axs = plt.subplots(ncols=len(imgs))
fig.set_size_inches((16, 10))
for i in range(len(imgs)):
    axs[i].imshow(imgs[i])
    axs[i].set_axis_off()

In [106]:
%%capture
gs = gridspec.GridSpec(2, 3, width_ratios=[1, 2, 1], height_ratios=[1, 1])  

fig = plt.figure(figsize=(18, 10))  
# Create subplots in specific grid locations
ax1 = fig.add_subplot(gs[0, 0])  # Top-left
ax2 = fig.add_subplot(gs[1, 0])  # Bottom-left
ax3 = fig.add_subplot(gs[:, 1])  # Middle column (spanning both rows)
ax4 = fig.add_subplot(gs[0, 2])  # Top-right
ax5 = fig.add_subplot(gs[1, 2])  # Bottom-right

plt.rcParams["font.family"] = "serif"
# inset_locs = ((0.215, 0.05, 0.3, 0.3), (0.275, 0.82, 0.3, 0.3),  (0.25, 0.445, 0.3, 0.3), (0.80, 0.74, 0.3, 0.3), (0.80, 0.74, 0.3, 0.3))
inset_locs = ((0.215, 0.05, 0.3, 0.3), (0.10, 0.60, 0.3, 0.3),  (0.1, 0.3, 0.3, 0.3), (0.80, 0.64, 0.3, 0.3), (0.24, 0.85, 0.3, 0.3))

TITLE_FS = 25
LABEL_FS = 23
TICK_FS = 21
IMG_SIZE = 0.35

ax1.imshow(img)
ax1.set_title('Image: (336x336)', fontsize=TITLE_FS)
ax1.set_axis_off()

ax2.imshow(im0)
ax2.set_title('DINOv2: (24x24)', fontsize=TITLE_FS)
ax2.set_axis_off()

ot, om = 0.2, 150
max_t = mem_time_results['Strided']['time'] + ot
max_m = mem_time_results['LoftUp']['mem'] + om

ax3.grid(True, linestyle="--", alpha=0.6)
ax3.set_xlim(-ot, max_t)
ax3.set_ylim(-om,  max_m)

ax3.set_xlabel('Time (s)', fontsize=LABEL_FS)
ax3.set_ylabel('Memory (MB)', fontsize=LABEL_FS)
ax3.tick_params(axis='both', labelsize=TICK_FS)

for i, res in enumerate(mem_time_results.items()):
    im = imgs[i]
    name, value = res
    x = value['time']# / max_t
    y = value['mem'] #/ max_m
    ax3.scatter(x, y, lw=10)

    o_img = OffsetImage(im, zoom=IMG_SIZE)
    img_size = int(IMG_SIZE * 336)
    loc = inset_locs[i]
    ix, iy = loc[0] * max_t, loc[1] * max_m
    ab = AnnotationBbox(o_img, (ix, iy), frameon=True, bboxprops=dict(edgecolor=f"C{i}", linewidth=8), pad=0)
    ax3.add_artist(ab)
ax3.legend(mem_time_results.keys(), fontsize=TICK_FS, loc='lower right')


ax4.set_xlabel('Image length (px)', fontsize=LABEL_FS)
ax4.set_ylabel('Time (s)', fontsize=LABEL_FS)
ax4.tick_params(axis='both', labelsize=TICK_FS)

for name, times in time_vs_l.items():
    ax4.plot(lengths[:len(times)], times, label=name, marker='.', lw=3, ms=15)
ax4.semilogy()
ax4.grid(True, linestyle="--", alpha=0.6)

ax5.set_xlabel('Image length (px)', fontsize=LABEL_FS)
ax5.set_ylabel('Memory (MB)', fontsize=LABEL_FS)
ax5.tick_params(axis='both', labelsize=TICK_FS)
ax5.grid(True, linestyle="--", alpha=0.6)

for name, mems in mem_vs_l.items():
    ax5.plot(lengths[:len(mems)], mems, label=name, marker='.', lw=3, ms=15)
ax5.semilogy()

for key, ax in zip(('a', 'b', 'c'), (ax1, ax3, ax4)):
    y = 1.016 if key =='b' else 1.05
    x = -0.26 if key =='c' else -0.15
    ax.text(x, y, f"{key}.", transform=ax.transAxes, 
            size=LABEL_FS + 4, weight='bold')

plt.tight_layout(pad=2.5)
plt.savefig('fig_out/perf_landscape.png')

In [101]:
plt.close()